# Creating & Uploading python distribution
https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container

In [112]:
!python setup.py sdist

running sdist
running egg_info
writing dat_package.egg-info\PKG-INFO
writing dependency_links to dat_package.egg-info\dependency_links.txt
writing requirements to dat_package.egg-info\requires.txt
writing top-level names to dat_package.egg-info\top_level.txt
reading manifest file 'dat_package.egg-info\SOURCES.txt'
writing manifest file 'dat_package.egg-info\SOURCES.txt'
running check
creating dat-package-0.1
creating dat-package-0.1\dat_package.egg-info
creating dat-package-0.1\trainer
creating dat-package-0.1\trainer\dat
copying files to dat-package-0.1...
copying README.md -> dat-package-0.1
copying setup.py -> dat-package-0.1
copying dat_package.egg-info\PKG-INFO -> dat-package-0.1\dat_package.egg-info
copying dat_package.egg-info\SOURCES.txt -> dat-package-0.1\dat_package.egg-info
copying dat_package.egg-info\dependency_links.txt -> dat-package-0.1\dat_package.egg-info
copying dat_package.egg-info\requires.txt -> dat-package-0.1\dat_package.egg-info
copying dat_package.egg-info\top

In [86]:
!python -m trainer.task

Traceback (most recent call last):
  File "c:\Users\caspa\miniconda3\envs\seminar_dml_dat\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\caspa\miniconda3\envs\seminar_dml_dat\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\caspa\miniconda3\envs\seminar_dml_dat\lib\site-packages\trainer\task.py", line 67, in <module>
    global_noise_data = torch.zeros([512, 3, 224, 224]).cuda()
  File "c:\Users\caspa\miniconda3\envs\seminar_dml_dat\lib\site-packages\torch\cuda\__init__.py", line 239, in _lazy_init
    raise AssertionError("Torch not compiled with CUDA enabled")
AssertionError: Torch not compiled with CUDA enabled


In [113]:
#!gcloud storage cp dist/dat_package-0.1.tar.gz gs://dat-package-bucket
!gsutil cp dist/dat-package-0.1.tar.gz gs://dat-project-bucket/dat-package/

Copying file://dist\dat-package-0.1.tar.gz [Content-Type=application/x-tar]...
/ [0 files][    0.0 B/ 14.9 KiB]                                                
/ [1 files][ 14.9 KiB/ 14.9 KiB]                                                

Operation completed over 1 objects/14.9 KiB.                                     


# Creating CustomJob
https://cloud.google.com/vertex-ai/docs/training/create-custom-job#create_custom_job-gcloud

# Operationalize Distributed Training with PyTorch on Google Cloud
https://www.youtube.com/watch?v=kXbDEmn2XOw

Pre-built-containers:
https://cloud.google.com/vertex-ai/docs/training/pre-built-containers#pytorch

GPUs:
https://cloud.google.com/compute/docs/gpus
https://cloud.google.com/compute/gpus-pricing
https://cloud.google.com/vertex-ai/docs/training/configure-compute#gpu-compatibility-table

In [ ]:
#!pip install google-cloud-aiplatform

In [114]:
from google.cloud import aiplatform

project_id = 'sdml-dat'
region = 'us-central1' #TODO: change to better region
bucket_name = 'dat-project-bucket'
package_uri = f'gs://{bucket_name}/dat-package/dat-package-0.1.tar.gz'

# Initialize Vertex AI SDK
aiplatform.init(project=project_id, location=region, staging_bucket=bucket_name)

# Define worker pool spec
worker_pool_specs =  [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
        },
        "replica_count": 1,  # Number of masters
        "python_package_spec": {
            "executor_image_uri": "europe-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-13.py310:latest",
            "package_uris": [package_uri],
            "python_module": "trainer.task",
            "args": [
                "--gcloud=True",
                "--output-dir=/gcs/dat-project-bucket/dat-outputs/"
            ],
        },
    }
    ,
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
            # "accelerator_type": "NVIDIA_TESLA_K80",
            # "accelerator_count": 2,
        },
        "replica_count": 2,  # Number of workers
        "python_package_spec": {
            "executor_image_uri": "europe-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-13.py310:latest",
            "package_uris": [package_uri],
            "python_module": "trainer.task",
            "args": [
                "--gcloud=True",
                "--output-dir=/gcs/dat-project-bucket/dat-outputs/"
            ],
        },
    },
]

# Define job name
job_name = 'dat-custom-pytorch-job'

# Create a custom job
job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
)

# Launch the job
job.run(sync=True)

print(f"Job {job_name} has been launched.")

Creating CustomJob
CustomJob created. Resource name: projects/65830871515/locations/us-central1/customJobs/311184742965837824
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/65830871515/locations/us-central1/customJobs/311184742965837824')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/311184742965837824?project=65830871515
CustomJob projects/65830871515/locations/us-central1/customJobs/311184742965837824 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/65830871515/locations/us-central1/customJobs/311184742965837824 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/65830871515/locations/us-central1/customJobs/311184742965837824 current state:
JobState.JOB_STATE_RUNNING
CustomJob projects/65830871515/locations/us-central1/customJobs/311184742965837824 current state:
JobState.JOB_STATE_RUNNING
CustomJob projects/65830871515/locations/us-central1/customJobs/311184742965837824 cur

RuntimeError: Job failed with:
code: 3
message: "The replica workerpool0-0 exited with a non-zero status of 1. Termination reason: Error. \nTraceback (most recent call last):\n  File \"/opt/conda/lib/python3.10/runpy.py\", line 196, in _run_module_as_main\n    return _run_code(code, main_globals, None,\n  File \"/opt/conda/lib/python3.10/runpy.py\", line 86, in _run_code\n    exec(code, run_globals)\n  File \"/root/.local/lib/python3.10/site-packages/trainer/task.py\", line 308, in <module>\n    main()\n  File \"/root/.local/lib/python3.10/site-packages/trainer/task.py\", line 236, in main\n    global_noise_data = torch.zeros([batch_size, 3, 32, 32]).cuda()\n  File \"/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py\", line 229, in _lazy_init\n    torch._C._cuda_init()\nRuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx\n\nThe replica workerpool1-0 exited with a